# Steam (Environment - Agent)

In [1]:
# Basic import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import norm
import pdb
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, Dot, Concatenate
from tensorflow.keras.models import Model
tf.config.experimental_run_functions_eagerly(True)
print(tf.__version__)

2.0.0


## Environment

In [2]:
class Environment:
    """ 
    Contextual Multi-Armed Bandit environment
    """
    def __init__(self, nb_films, nb_users, 
                 context_size = 2,
                 displayed_users_embedding_size = 2, #used for the features vector
                 displayed_games_embedding_size = 2, #used for the features vector
                 noise_size = 3,
                 seed=None):     
        
        self._rng = np.random.RandomState(seed)
        #-------------------------------------------------------#
        
        self._nb_games = nb_games
        self._nb_users = nb_users
        self._p = context_size # size of user, size of game
        self._displayed_users_embedding_size = displayed_users_embedding_size
        self._displayed_games_embedding_size = displayed_games_embedding_size
        self._noise_size = noise_size
   
        #-------------------------------------------------------#
    
        self.user_mean = np.ones(self._p)
        self.user_var = np.ones(self._p)
        self.game_mean = np.ones(self._p)
        self.game_var = np.ones(self._p)
        
        #-------------------------------------------------------#
        self.finish = False # flag to know when reset the environment (all games played)
    
    def step(self):
        
        if self.finish:
            print("All games played reset the environment")
            
        """ Choose a game """
        user = self._rng.randint(0, self._nb_users)
        available_games = np.where(self._available_games[user] == 1)[0]
        return user, available_games
    
    def update(self, user, game):
        reward = self._reward_matrix[user, game]
        self._available_games[user, game] = 0
        return reward
    
    def reset(self):
        self._users = self._rng.normal(loc=self.user_mean,
                                                scale=self.user_var,
                                                size=(self._nb_users, self._p))
        self._games = self._rng.normal(loc=self.game_mean,
                                                scale=self.game_var,
                                                size=(self._nb_games, self._p))
        
        z_mean = self.user_mean.dot(self.game_mean)
        z_var = self.user_var.dot(self.game_var) + self.user_var.dot(np.square(self.game_mean)) + \
                self.game_var.dot(np.square(self.user_mean))
        z = norm(z_mean, np.sqrt(z_var))
        self.z_cut_points = z.ppf([0.2, 0.4, 0.6, 0.8]) # buckets
        self._available_games = np.ones((nb_users, nb_games))        
        
        self._reward_matrix = np.zeros((nb_users, nb_games))
        
        for i in range(self._reward_matrix.shape[0]):
            for j in range(self._reward_matrix.shape[1]):
                real_score = self._users[i].dot(self._games[j])
                self._reward_matrix[i, j] = np.searchsorted(self.z_cut_points, real_score) + 1

        users = deepcopy(self._users)
        return users

    def get_feature_vector(self, user, game):
        user_embedding = self._users[user]
        game_embedding = self._games[game]
        
        if self._displayed_users_embedding_size + self._displayed_games_embedding_size > 0:
            variables = np.array([user_embedding[:self._displayed_users_embedding_size],
                                  game_embedding[:self._displayed_games_embedding_size]])

            if self._noise_size > 0:
                noise = self._rng.normal(loc=np.ones(self._noise_size),
                                         scale=np.ones(self._noise_size),
                                         size=self._noise_size)
                
                variables = np.append(variables, noise)
                
            return variables

## Agent

In [3]:
class RandomAgent:
    """ 
    Random agent
    """
    def __init__(self, seed = None):
        self._rng = np.random.RandomState(seed)
    
    def act(self, available_games):
        action = self._rng.choice(available_games)
        return action

## Experiment

In [4]:
# Basic parameter
nb_users = 30 #number of users in the context
nb_games = 10 #number of games in the context
context_size = 2 #number of different film categories = 2

In [5]:
# Creating the environment
env = Environment(nb_games,nb_users,context_size,seed=2020)
env.reset() #reset and initilize the environment

array([[-0.76884571,  1.07555227],
       [-0.1306297 ,  0.34856983],
       [ 0.10688437, -0.27410098],
       [ 0.93884557,  1.06451384],
       [ 1.41011295,  0.42711751],
       [ 0.19866638,  2.31203519],
       [ 2.27469887, -0.2143576 ],
       [ 1.31371941, -0.44482142],
       [ 0.6310387 ,  0.23077342],
       [ 1.3926161 ,  1.05729383],
       [ 3.08997884,  1.04197131],
       [ 0.95165928,  0.48684608],
       [ 0.91541072, -0.21545008],
       [-0.41293073, -0.48691055],
       [ 1.38222486,  1.937673  ],
       [ 2.77267804,  1.87882801],
       [ 1.33171912,  0.69396433],
       [ 2.24026615,  0.78437316],
       [ 1.15592948,  1.09805553],
       [ 1.83209585,  3.04520542],
       [ 0.68318608, -0.31283291],
       [-0.75445746,  1.10209408],
       [-0.36150208,  1.48178488],
       [ 0.79167126,  0.90813649],
       [ 1.70268816,  1.10365506],
       [ 1.62123638,  1.95411497],
       [ 3.03781352,  0.51554878],
       [ 1.2071549 ,  2.64424216],
       [ 0.5117926 ,

In [6]:
# Creating the agent
agent = RandomAgent(2020)

We run the experiment and generate some historical data.

In [7]:
# Running several trials
nb_iteration = 100 #how many trials
rating_matrix = np.zeros((env._nb_users, env._nb_games))
users = list()
games = list()
ratings = list()
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(available_games)
    reward = env.update(user, choosen_game)
    users.append(user)
    games.append(choosen_game)
    ratings.append(reward)
    rating_matrix[user, choosen_game] = reward
    '''
    print("user = {}, recommended_games = {}, choosen_game = {}".format(user,recommended_games,choosen_game))
    print("reward = {}\n".format(reward))
    '''
    
print("rating matrix: \n", str(rating_matrix))

rating matrix: 
 [[0. 0. 0. 3. 0. 0. 0. 2. 2. 0.]
 [0. 0. 1. 2. 1. 0. 0. 0. 0. 1.]
 [0. 0. 2. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
 [0. 3. 4. 2. 0. 0. 0. 3. 3. 2.]
 [0. 3. 0. 0. 0. 0. 3. 0. 5. 1.]
 [2. 0. 5. 1. 5. 3. 0. 0. 3. 3.]
 [0. 2. 4. 0. 0. 2. 0. 0. 0. 0.]
 [2. 2. 2. 0. 0. 2. 0. 2. 2. 2.]
 [2. 0. 0. 2. 5. 4. 0. 0. 4. 2.]
 [0. 4. 0. 0. 5. 0. 0. 4. 0. 0.]
 [0. 2. 3. 2. 0. 0. 0. 0. 0. 0.]
 [2. 0. 3. 0. 4. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 4. 5. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 5. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 4. 0. 0. 0. 0. 0. 4. 0. 0.]
 [2. 0. 3. 0. 0. 0. 0. 0. 0. 0.]
 [3. 5. 0. 4. 0. 0. 0. 5. 5. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 1. 3. 2. 0. 3. 0.]
 [0. 0. 3. 0. 0. 0. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
 [0. 0. 0. 3. 5. 0. 2. 4. 0. 0.]
 [0. 0. 0. 0. 0. 4. 1. 0. 4. 0.]
 [3. 0. 0. 4. 4. 0. 2. 0. 5. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 3. 0.]
 [0. 3. 4. 2. 0. 0. 0. 4. 

## Regression model

In [8]:
class RegressionModel(Model):
    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        self.flatten = Flatten()
        self.dot = Dot(axes=1)
        
    def call(self, inputs):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        y = self.dot([user_vecs, game_vecs])
        return y

## Deep regression model

In [9]:
class DeepRegressionModel(Model):

    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        
        self.flatten = Flatten()
        self.concat = Concatenate()
        
        self.dense1 = Dense(16, activation="relu")
        self.dense2 = Dense(8, activation="relu")
        
    def call(self, inputs, training=False):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        feature_inputs = inputs[2]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        # input_vecs = self.concat([user_vecs, game_vecs, self.flatten(feature_inputs)])
        input_vecs = self.concat([user_vecs, game_vecs])
        
        y = self.dense1(input_vecs)
        y = self.dense2(y)
        
        return y

## Embedding agent

In [10]:
class EmbeddingAgent:
    def __init__(self, X, Y, deepRegression=False):
        if deepRegression:
            self._model = DeepRegressionModel(64, nb_users, nb_games) ## passare nb
        else:
            self._model = RegressionModel(64, nb_users, nb_games)
        self._model.compile(optimizer="adam", loss='mae')
        self._model.fit(X, Y,
                  batch_size=64, epochs=100, validation_split=0.1,
                  shuffle=True)
        self._user_embeddings = self._model.get_weights()[0]
        self._game_embeddings = self._model.get_weights()[1]
    
    def act(self, user, available_games):
        user_embedding = self._user_embeddings[user]
        dot_products = self._game_embeddings @ user_embedding
        user_embedding_norm = np.linalg.norm(user_embedding)
        all_item_norms = np.linalg.norm(self._game_embeddings, axis=1)
        norm_products = user_embedding_norm * all_item_norms
        sims = dot_products / (norm_products)
        sims = np.argsort(sims)[::-1]
        mask = np.in1d(sims, available_games)
        sims = sims[mask]
        return sims[0]

In [11]:
deepRegression = True

users = np.array(users)
games = np.array(games)
ratings = np.array(ratings)

if deepRegression:
    features = []
    for i in range(len(users)):
        features.append(env.get_feature_vector(users[i], games[i]))
    features = np.float64(features)
    agent = EmbeddingAgent([users, games, features], ratings, deepRegression=deepRegression)
else:
    agent = EmbeddingAgent([users, games], ratings, deepRegression=deepRegression)

Train on 90 samples, validate on 10 samples
Epoch 1/100
90/90 [==============================] - 0s 2ms/sample - loss: 2.7536 - val_loss: 2.9734
Epoch 2/100
90/90 [==============================] - 0s 573us/sample - loss: 2.7467 - val_loss: 2.9654
Epoch 3/100
90/90 [==============================] - 0s 539us/sample - loss: 2.7394 - val_loss: 2.9571
Epoch 4/100
90/90 [==============================] - 0s 579us/sample - loss: 2.7315 - val_loss: 2.9485
Epoch 5/100
90/90 [==============================] - 0s 554us/sample - loss: 2.7231 - val_loss: 2.9391
Epoch 6/100
90/90 [==============================] - 0s 601us/sample - loss: 2.7140 - val_loss: 2.9292
Epoch 7/100
90/90 [==============================] - 0s 716us/sample - loss: 2.7044 - val_loss: 2.9188
Epoch 8/100
90/90 [==============================] - 0s 621us/sample - loss: 2.6940 - val_loss: 2.9077
Epoch 9/100
90/90 [==============================] - 0s 572us/sample - loss: 2.6828 - val_loss: 2.8956
Epoch 10/100
90/90 [===========

90/90 [==============================] - 0s 554us/sample - loss: 0.5031 - val_loss: 0.8302
Epoch 80/100
90/90 [==============================] - 0s 547us/sample - loss: 0.5009 - val_loss: 0.8408
Epoch 81/100
90/90 [==============================] - 0s 550us/sample - loss: 0.4985 - val_loss: 0.8544
Epoch 82/100
90/90 [==============================] - 0s 556us/sample - loss: 0.4960 - val_loss: 0.8654
Epoch 83/100
90/90 [==============================] - 0s 542us/sample - loss: 0.4946 - val_loss: 0.8701
Epoch 84/100
90/90 [==============================] - 0s 562us/sample - loss: 0.4923 - val_loss: 0.8751
Epoch 85/100
90/90 [==============================] - 0s 483us/sample - loss: 0.4901 - val_loss: 0.8774
Epoch 86/100
90/90 [==============================] - 0s 487us/sample - loss: 0.4883 - val_loss: 0.8784
Epoch 87/100
90/90 [==============================] - 0s 506us/sample - loss: 0.4875 - val_loss: 0.8775
Epoch 88/100
90/90 [==============================] - 0s 530us/sample - loss:

In [12]:
# Running several trials
nb_iteration = 20 #how many trials
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(user, available_games)
    reward = env.update(user, choosen_game)
    rating_matrix[user, choosen_game] = reward
    print("user = {}, available games = {}, choosen_game = {}".format(user,available_games,choosen_game))
    print("reward = {}\n".format(reward))

user = 21, available games = [1 3 4 5 6 7 8 9], choosen_game = 4
reward = 1.0

user = 2, available games = [0 1 3 4 5 8 9], choosen_game = 5
reward = 1.0

user = 0, available games = [0 1 2 4 5 6 9], choosen_game = 5
reward = 2.0

user = 22, available games = [0 2 3 7 9], choosen_game = 7
reward = 3.0

user = 27, available games = [1 2 5 7], choosen_game = 5
reward = 5.0

user = 7, available games = [0 3 4 6 7 8 9], choosen_game = 8
reward = 2.0

user = 3, available games = [0 1 2 3 4 6 7 8 9], choosen_game = 4
reward = 4.0

user = 20, available games = [0 1 2 4 5 6 7 8 9], choosen_game = 0
reward = 2.0

user = 11, available games = [0 4 5 6 7 8 9], choosen_game = 5
reward = 3.0

user = 27, available games = [1 2 7], choosen_game = 2
reward = 3.0

user = 7, available games = [0 3 4 6 7 9], choosen_game = 7
reward = 2.0

user = 1, available games = [0 1 5 6 7 8], choosen_game = 1
reward = 2.0

user = 0, available games = [0 1 2 4 6 9], choosen_game = 4
reward = 1.0

user = 2, available 